In [11]:
#https://medium.com/codestorm/scrape-yahoo-finance-using-pandas-d52f502b7289

In [1]:
import pandas as pd
import requests
import numpy as np

In [23]:
ticker = "MSFT"

In [24]:
#gets data
def getdata(url):
  r = requests.get(url,headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
  data = pd.read_html(r.text)
  return data

In [25]:
#scrapes stock summary from Yahoo Finance
summary_url = f'https://finance.yahoo.com/quote/{ticker}?p={ticker}'
summary_data = getdata(summary_url)
#formatting
data = [summary_data[0], summary_data[1]]
data = pd.concat(summary_data)
data.reset_index(drop=True, inplace=True)
df = data.transpose()
df.columns = df.iloc[0]
df = df.drop(0)
df = df.reset_index(drop=True)

In [28]:
#scrape stock historical data from Yahoo Finance
historiscal_url= f'https://finance.yahoo.com/quote/{ticker}/history?p={ticker}'
historiscal_data = getdata(historiscal_url)
data = historiscal_data[0]
#formatting
data = data.iloc[:-1 , :]
data = data[data["Open"].str.contains("Dividend") == False]

In [31]:
data['Date'] = pd.to_datetime(data['Date'])
numeric_columns = list(data.columns)[1::]
for column_name in numeric_columns:
    data[column_name] = data[column_name].str.replace(',', '')
    data[column_name] = data[column_name].astype(np.float64)

In [32]:
#scrapes stock statistics from Yahoo Finance
statistics_url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
statistics_data = getdata(statistics_url)

In [33]:
#defines each statistic as its own index
valuation_Measures = statistics_data[0]
stock_Price_History = statistics_data[1]
share_Statistics = statistics_data[2]
dividend_Info = statistics_data[3]
fiscal_year = statistics_data[4]
profitability_Info = statistics_data[5]
management_Efectiveness = statistics_data[6]
income_Statement = statistics_data[7]
balance_Sheet = statistics_data[8]
cash_Flow = statistics_data[9]

In [36]:
data = valuation_Measures
df = data.transpose()
df = df.reset_index()
df.columns = df.iloc[0]
df = df.drop(0)
df = df.rename(columns={"Unnamed: 0": "Date"})

In [37]:
df

,Date,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA
1,As of Date: 3/8/2023Current,1.91T,1.87T,28.54,23.75,1.93,9.44,10.44,9.18,18.96
2,12/31/2022,1.79T,1.74T,25.81,25.19,2.12,8.88,10.30,33.00,71.12
3,9/30/2022,1.74T,1.69T,24.13,23.09,1.73,8.86,10.43,33.78,68.11
4,6/30/2022,1.92T,1.87T,26.81,23.81,1.73,10.08,11.77,36.12,75.06
5,3/31/2022,2.31T,2.25T,32.83,28.57,2.12,12.63,14.42,45.50,91.79
6,12/31/2021,2.52T,2.46T,37.62,36.90,2.87,14.48,16.60,47.46,92.52
